**Important note!** Before you turn in this lab notebook, make sure everything runs as expected:

- First, **restart the kernel** -- in the menubar, select Kernel$\rightarrow$Restart.
- Then **run all cells** -- in the menubar, select Cell$\rightarrow$Run All.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

# Lesson 2: NYC 311 calls

This notebook derives from a [demo by the makers of plot.ly](https://plot.ly/ipython-notebooks/big-data-analytics-with-pandas-and-sqlite/). We've adapted it to use [Bokeh (and HoloViews)](http://bokeh.pydata.org/en/latest/).

You will start with a large database of complaints filed by residents of New York City since 2010 via 311 calls. The full dataset is available at the [NYC open data portal](https://nycopendata.socrata.com/data). At about 6 GB and 10 million complaints, you can infer that a) you might not want to read it all into memory at once, and b) NYC residents have a lot to complain about. (Maybe only conclusion "a" is valid.) The notebook then combines the use of `sqlite`, `pandas`, and `bokeh`.

## Module setup

Before diving in, run the following cells to preload some functions you'll need later.

In [ ]:
from IPython.display import display
import pandas as pd

We'll also need some functionality from earlier notebooks.

In [ ]:
def canonicalize_tibble(X):
    var_names = sorted(X.columns)
    Y = X[var_names].copy()
    Y.sort_values(by=var_names, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    return Y

def tibbles_are_equivalent (A, B):
    A_canonical = canonicalize_tibble(A)
    B_canonical = canonicalize_tibble(B)
    cmp = A_canonical.eq(B_canonical)
    return cmp.all().all()

def cast(df, key, value, join_how='outer'):
    """Casts the input data frame into a tibble,
    given the key column and value column.
    """
    assert type(df) is pd.DataFrame
    assert key in df.columns and value in df.columns
    assert join_how in ['outer', 'inner']
    
    fixed_vars = df.columns.difference([key, value])
    tibble = pd.DataFrame(columns=fixed_vars) # empty frame    
    new_vars = df[key].unique()
    for v in new_vars:
        df_v = df[df[key] == v]
        del df_v[key]
        df_v = df_v.rename(columns={value: v})
        tibble = tibble.merge(df_v,
                              on=list(fixed_vars),
                              how=join_how)    
    return tibble

Lastly, some of the test cells will need some auxiliary files, which the following code cell will check for and, if they are missing, download.

In [ ]:
import requests
import os
import hashlib
import io

def download(file, url_suffix=None, checksum=None):
    if url_suffix is None:
        url_suffix = file
        
    if os.path.exists('.voc'):
        url = 'https://cse6040.gatech.edu/datasets/{}'.format(url_suffix)
    else:
        url = 'https://github.com/cse6040/labs-fa17/raw/master/{}'.format(url_suffix)
    if os.path.exists(file):
        print("[{}]\n==> '{}' is already available.".format(url, file))
    else:
        print("[{}] Downloading...".format(url))
        r = requests.get(url)
        with open(file, 'w', encoding=r.encoding) as f:
            f.write(r.text)
            
    if checksum is not None:
        with io.open(file, 'r', encoding='utf-8', errors='replace') as f:
            body = f.read()
            body_checksum = hashlib.md5(body.encode('utf-8')).hexdigest()
            assert body_checksum == checksum, \
                "Downloaded file '{}' has incorrect checksum: '{}' instead of '{}'".format(file, body_checksum, checksum)
            print("==> Checksum test passes: {}".format(checksum))
    
    print("==> '{}' is ready!\n".format(file))
    
auxfiles = {'df_complaints_by_city_soln.csv': '2a82e5856d5a267db9aafc26f16c3ae1',
            'df_complaints_by_hour_soln.csv': 'f06fcd917876d51ad52ddc13b2fee69e',
            'df_noisy_by_hour_soln.csv': '30f3fa7c753d4d3f4b3edfa1f6d05bcc',
            'df_plot_stacked_fraction_soln.csv': '2ca04a3eb24ccc37ddd0f8f5917fb27a'}

for filename, checksum in auxfiles.items():
    download(filename, url_suffix='{}/{}'.format('lab9-sql', filename), checksum=checksum)
    
print("(Auxiliary files appear to be ready.)")

## Viz setup

This notebook includes some simple visualizations. This section just ensures you have the right software setup to follow along.

In [ ]:
# Build a Pandas data frame
names = ['Bob','Jessica','Mary','John','Mel']
births = [968, 155, 77, 578, 973]
name_birth_pairs = list(zip(names, births))
baby_names = pd.DataFrame(data=name_birth_pairs, columns=['Names', 'Births'])
display(baby_names)

In [ ]:
import holoviews as hv # Replacement for bokeh.charts / bkcharts
hv.extension('bokeh')
from holoviews import Bars

In [ ]:
%%opts Bars [width=640 height=320]
Bars(baby_names, kdims=['Names'], vdims=['Births'], color='Names')

In addition to the HoloViews interface (above), some of the visualizations will use the Bokeh mid-level interface.

In [ ]:
from bokeh.io import show, output_notebook
output_notebook()

In [ ]:
# Adapted from: https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html#userguide-categorical
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.core.properties import value

def make_stacked_bar(df, x_var, bar_vars, kwargs_figure={}):
    assert type(x_var) is str, "x-variable should be a string but isn't."
    assert all([b in df.columns for b in bar_vars]), "Data frame is missing one or more columns: {}".format(bar_vars)
    
    from bokeh.palettes import brewer
    assert len(bar_vars) in brewer['Dark2'], "Not enough colors."
    
    x = list(df[x_var])
    colors = brewer['Dark2'][len(bar_vars)]
    legend = [value(b) for b in bar_vars]
    source = ColumnDataSource(data=df)

    p = figure(x_range=x, **kwargs_figure)
    p.vbar_stack(bar_vars, x=x_var, width=0.9,
             fill_color=colors, line_color=None,
             legend=legend,
             source=source)
    return p

## Data setup

You'll also need the NYC 311 calls dataset. What we've provided is actually a small subset (about 250+ MiB) of the full data as of 2015.

In [ ]:
import requests
import os
import hashlib
import io

def on_vocareum():
    return os.path.exists('.voc')

if on_vocareum():
    DB_FILENAME = None # TBD
else:
    DB_FILENAME = 'NYC-311-2M.db'
    
if not os.path.exists(DB_FILENAME):
    url = 'https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDE'
    print("Downloading: {} ...".format(url))
    r = requests.get(url)
    with open(file, 'w', encoding=r.encoding) as f:
        f.write(r.text)

DB_CHECKSUM = 'f48eba2fb06e8ece7479461ea8c6dee9'
with io.open(DB_FILENAME, 'rb') as f:
    body = f.read()
    body_checksum = hashlib.md5(body).hexdigest()
    assert body_checksum == DB_CHECKSUM, \
        "Database file '{}' has an incorrect checksum: '{}' instead of '{}'".format(DB_FILENAME,
                                                                                    body_checksum,
                                                                                    DB_CHECKSUM)
    
print("'{}' is ready!".format(DB_FILENAME))  
print("\n(All data appears to be ready.)")

**Connecting.** Let's open up a connection to this dataset.

In [ ]:
# Connect
import sqlite3 as db
disk_engine = db.connect(DB_FILENAME)

**Preview the data.** This sample database has just a single table, named `data`. Let's query it and see how long it takes to read. To carry out the query, we will use the SQL reader built into `pandas`.

In [ ]:
import time

print ("Reading ...")
start_time = time.time ()

# Perform SQL query through the disk_engine connection.
# The return value is a pandas data frame.
df = pd.read_sql_query ('select * from data', disk_engine)

elapsed_time = time.time () - start_time
print ("==> Took %g seconds." % elapsed_time)

# Dump the first few rows
df.head()

**Partial queries: `LIMIT` clause.** The preceding command was overkill for what we wanted, which was just to preview the table. Instead, we could have used the `LIMIT` option to ask for just a few results.

In [ ]:
query = '''
  select *
    from data
    limit 5
'''
start_time = time.time ()
df = pd.read_sql_query (query, disk_engine)
elapsed_time = time.time () - start_time
print ("==> LIMIT version took %g seconds." % elapsed_time)

df

**Grouping Information: GROUP BY operator.** The GROUP BY operator lets you group information using a particular column or multiple columns of the table. The output generated is more of a pivot table.

In [ ]:
query = '''
  select ComplaintType, Descriptor, Agency
    from data
    GROUP BY ComplaintType
    limit 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

In [ ]:
query = '''
  select ComplaintType, Descriptor, Agency
    from data
    limit 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

**Set membership: `IN` operator.** Another common idiom is to ask for rows whose attributes fall within a set, for which you can use the `IN` operator.

In [ ]:
query = '''
  select ComplaintType, Descriptor, Agency
    from data
    where Agency IN ("NYPD", "DOB")
    limit 10
'''

df = pd.read_sql_query (query, disk_engine)
df.head ()

**Finding unique values: `DISTINCT` qualifier.** Yet another common idiom is to ask for the unique values of some attribute, for which you can use the `DISTINCT` qualifier.

In [ ]:
query = 'select DISTINCT City FROM data'
df = pd.read_sql_query(query, disk_engine)

print("Found {} unique cities. The first few are:".format(len(df)))
df.head()

**Renaming columns: `AS` operator.** Sometimes you might want to rename a result column. For instance, the following query counts the number of complaints by "Agency," using the `COUNT(*)` function and `GROUP BY` clause, which we discussed in an earlier lab. If you wish to refer to the counts column of the resulting data frame, you can give it a more "friendly" name using the `AS` operator.

In [ ]:
query = '''
  select Agency, count(*) as NumComplaints
    from data
    group by Agency
'''
df = pd.read_sql_query(query, disk_engine)
df.head()

**Ordering results: `ORDER` clause.** You can also order the results. For instance, suppose we want to execute the previous query by number of complaints.

In [ ]:
query = '''
  select Agency, count(*) as NumComplaints
    from data
    group by Agency
    order by NumComplaints
'''
df = pd.read_sql_query(query, disk_engine)
df.tail()

Note that the above example prints the bottom (tail) of the data frame. You could have also asked for the query results in reverse (descending) order, by prefixing the `ORDER BY` attribute with a `-` (minus) symbol.

In [ ]:
query = '''
  select Agency, count(*) as NumComplaints
    from data
    group by Agency
    order by -NumComplaints
'''
df = pd.read_sql_query(query, disk_engine)
df.head()

And of course we can plot all of this data!

**Exercise 0** (ungraded). Run the following code cell, which will create an interactive bar chart from the data in the previous query.

In [ ]:
%%opts Bars [width=640 height=320]
chart = Bars(df[:20], kdims=['Agency'], vdims=[('NumComplaints', '# complaints')],
             label='Top 20 agencies by number of complaints')
chart(plot={'xrotation': 25})

**Exercise 1** (2 points). Create a string, `query`, containing an SQL query that will return the number of complaints by type. The columns should be named `type` and `freq`, and the results should be sorted in descending order by `freq`.

> What is the most common type of complaint? What, if anything, does it tell you about NYC?

In [ ]:
del query

# Define a variable named `query` containing your solution
# YOUR CODE HERE
raise NotImplementedError()

# Runs your `query`:
df_complaint_freq = pd.read_sql_query(query, disk_engine)
df_complaint_freq.head()

In [ ]:
# Test cell: `complaints_test`

print("Top 10 complaints:")
display(df_complaint_freq.head(10))

assert set(df_complaint_freq.columns) == {'type', 'freq'}, "Output columns should be named 'type' and 'freq', not {}".format(set(df_complaint_freq.columns))

soln = ['HEAT/HOT WATER', 'Street Condition', 'Street Light Condition', 'Blocked Driveway', 'Illegal Parking', 'UNSANITARY CONDITION', 'PAINT/PLASTER', 'Water System', 'PLUMBING', 'Noise', 'Noise - Street/Sidewalk', 'Traffic Signal Condition', 'Noise - Commercial', 'DOOR/WINDOW', 'WATER LEAK', 'Dirty Conditions', 'Sewer', 'Sanitation Condition', 'DOF Literature Request', 'ELECTRIC', 'Rodent', 'FLOORING/STAIRS', 'General Construction/Plumbing', 'Building/Use', 'Broken Muni Meter', 'GENERAL', 'Missed Collection (All Materials)', 'Benefit Card Replacement', 'Derelict Vehicle', 'Noise - Vehicle', 'Damaged Tree', 'Consumer Complaint', 'Derelict Vehicles', 'Taxi Complaint', 'Overgrown Tree/Branches', 'Graffiti', 'Snow', 'Opinion for the Mayor', 'APPLIANCE', 'Maintenance or Facility', 'Animal Abuse', 'Dead Tree', 'HPD Literature Request', 'Root/Sewer/Sidewalk Condition', 'SAFETY', 'Elevator', 'Food Establishment', 'SCRIE', 'Air Quality', 'Agency Issues', 'Construction', 'Highway Condition', 'Other Enforcement', 'Water Conservation', 'Sidewalk Condition', 'Indoor Air Quality', 'Street Sign - Damaged', 'Traffic', 'Plumbing', 'Fire Safety Director - F58', 'Homeless Person Assistance', 'Homeless Encampment', 'Special Enforcement', 'Street Sign - Missing', 'Noise - Park', 'Vending', 'For Hire Vehicle Complaint', 'Food Poisoning', 'Special Projects Inspection Team (SPIT)', 'Hazardous Materials', 'Electrical', 'DOT Literature Request', 'Litter Basket / Request', 'Taxi Report', 'Illegal Tree Damage', 'DOF Property - Reduction Issue', 'Unsanitary Animal Pvt Property', 'Asbestos', 'Lead', 'Vacant Lot', 'DCA / DOH New License Application Request', 'Street Sign - Dangling', 'Smoking', 'Violation of Park Rules', 'OUTSIDE BUILDING', 'Animal in a Park', 'Noise - Helicopter', 'School Maintenance', 'DPR Internal', 'Boilers', 'Industrial Waste', 'Sweeping/Missed', 'Overflowing Litter Baskets', 'Non-Residential Heat', 'Curb Condition', 'Drinking', 'Standing Water', 'Indoor Sewage', 'Water Quality', 'EAP Inspection - F59', 'Derelict Bicycle', 'Noise - House of Worship', 'DCA Literature Request', 'Recycling Enforcement', 'ELEVATOR', 'DOF Parking - Tax Exemption', 'Broken Parking Meter', 'Request for Information', 'Taxi Compliment', 'Unleashed Dog', 'Urinating in Public', 'Unsanitary Pigeon Condition', 'Investigations and Discipline (IAD)', 'Bridge Condition', 'Ferry Inquiry', 'Bike/Roller/Skate Chronic', 'Public Payphone Complaint', 'Vector', 'BEST/Site Safety', 'Sweeping/Inadequate', 'Disorderly Youth', 'Found Property', 'Mold', 'Senior Center Complaint', 'Fire Alarm - Reinspection', 'For Hire Vehicle Report', 'City Vehicle Placard Complaint', 'Cranes and Derricks', 'Ferry Complaint', 'Illegal Animal Kept as Pet', 'Posting Advertisement', 'Harboring Bees/Wasps', 'Panhandling', 'Scaffold Safety', 'OEM Literature Request', 'Plant', 'Bus Stop Shelter Placement', 'Collection Truck Noise', 'Beach/Pool/Sauna Complaint', 'Complaint', 'Compliment', 'Illegal Fireworks', 'Fire Alarm - Modification', 'DEP Literature Request', 'Drinking Water', 'Fire Alarm - New System', 'Poison Ivy', 'Bike Rack Condition', 'Emergency Response Team (ERT)', 'Municipal Parking Facility', 'Tattooing', 'Unsanitary Animal Facility', 'Animal Facility - No Permit', 'Miscellaneous Categories', 'Misc. Comments', 'Literature Request', 'Special Natural Area District (SNAD)', 'Highway Sign - Damaged', 'Public Toilet', 'Adopt-A-Basket', 'Ferry Permit', 'Invitation', 'Window Guard', 'Parking Card', 'Illegal Animal Sold', 'Stalled Sites', 'Open Flame Permit', 'Overflowing Recycling Baskets', 'Highway Sign - Missing', 'Public Assembly', 'DPR Literature Request', 'Fire Alarm - Addition', 'Lifeguard', 'Transportation Provider Complaint', 'DFTA Literature Request', 'Bottled Water', 'Highway Sign - Dangling', 'DHS Income Savings Requirement', 'Legal Services Provider Complaint', 'Foam Ban Enforcement', 'Tunnel Condition', 'Calorie Labeling', 'Fire Alarm - Replacement', 'X-Ray Machine/Equipment', 'Sprinkler - Mechanical', 'Hazmat Storage/Use', 'Tanning', 'Radioactive Material', 'Rangehood', 'SRDE', 'Squeegee', 'Building Condition', 'SG-98', 'Standpipe - Mechanical', 'AGENCY', 'Forensic Engineering', 'Public Assembly - Temporary', 'VACANT APARTMENT', 'Laboratory', 'SG-99']
assert all(soln == df_complaint_freq['type'])

print("\n(Passed.)")

Let's also visualize the result, as a bar chart showing complaint types on the x-axis and the number of complaints on the y-axis.

In [ ]:
%%opts Bars [width=800 height=320]
chart = Bars(df_complaint_freq[:25], kdims=['type'], vdims=[('freq', '# complaints')],
             label='Top 25 complaints by type')
chart(plot={'xrotation': 25})

# Lesson 3: More SQL stuff

**Simple substring matching: the `LIKE` operator.** Suppose we just want to look at the counts for all complaints that have the word `noise` in them. You can use the `LIKE` operator combined with the string wildcard, `%`, to look for case-insensitive substring matches.

In [ ]:
query = '''
  select ComplaintType as type, count(*) as freq
    from data
    where ComplaintType like '%noise%'
    group by type
    order by -freq
'''

df_noisy = pd.read_sql_query(query, disk_engine)
df_noisy

**Exercise 2** (2 points). Create a string variable, `query`, that contains an SQL query that will return the top 10 cities with the largest number of complaints, in descending order. It should return a table with two columns, one named `name` holding the name of the city, and one named `freq` holding the number of complaints by that city.

In [ ]:
del query

# Define your `query`, here:
# YOUR CODE HERE
raise NotImplementedError()

# Runs your `query`:
df_whiny_cities = pd.read_sql_query(query, disk_engine)
df_whiny_cities

In [ ]:
# Test cell: `whiny_cities__test`

assert df_whiny_cities['name'][0] == 'BROOKLYN'
assert df_whiny_cities['name'][1] == 'NEW YORK'
assert df_whiny_cities['name'][2] == 'BRONX'
assert df_whiny_cities['name'][3] is None
assert df_whiny_cities['name'][4] == 'STATEN ISLAND'

print ("\n(Passed partial test.)")

You should notice two odd bits: cities are treated in a _case-sensitive_ manner and `None` appears as a city. (Presumably this setting occurs when a complaint is non-localized or the city is not otherwise specified.)

**Case-insensitive grouping: `COLLATE NOCASE`.** One way to carry out the preceding query in a case-insensitive way is to add a `COLLATE NOCASE` qualifier to the `GROUP BY` clause.

Let's filter out the 'None' cases as well, while we are at it.

In [ ]:
query = '''
  SELECT City as name, COUNT(*) AS freq
    FROM data
    WHERE name <> 'None'
    GROUP BY name COLLATE NOCASE
    ORDER BY -freq
    LIMIT 10
'''
df_whiny_cities2 = pd.read_sql_query(query, disk_engine)
df_whiny_cities2

Brooklynites are complainers, evidently.

Lastly, for later use, let's save the names of just the top seven (7) cities by numbers of complaints.

In [ ]:
TOP_CITIES = list(df_whiny_cities2.head(7)['name'])
TOP_CITIES

**Exercise 3** (1 point). Implement a function that takes a list of strings, `str_list`, and returns a single string consisting of each value, `str_list[i]`, enclosed by double-quotes and separated by a comma-space delimiters. For example, if

```python
   assert str_list == ['a', 'b', 'c', 'd']
```

then

```python
   assert strs_to_args(str_list) == '"a", "b", "c", "d"'
```

In [ ]:
def strs_to_args(str_list):
    assert type (str_list) is list
    assert all ([type (s) is str for s in str_list])
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test cell: `strs_to_args__test`

print ("Your solution, applied to TOP_CITIES:", strs_to_args(TOP_CITIES))

TOP_CITIES_as_args = strs_to_args(TOP_CITIES)
assert TOP_CITIES_as_args.lower() == \
       '"BROOKLYN", "NEW YORK", "BRONX", "STATEN ISLAND", "Jamaica", "Flushing", "ASTORIA"'.lower ()
    
print ("\n(Passed.)")

**Exercise 4** (3 points). Suppose we want to look at the number of complaints by type _and_ by city. Execute an SQL query to produce a tibble named `df_complaints_by_city` with the variables {`complaint_type`, `city_name`, `complaint_count`}.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test cell: `df_complaints_by_city__test`

print("Reading instructor's solution...")
df_complaints_by_city_soln = pd.read_csv('df_complaints_by_city_soln.csv')

print("Checking...")
assert tibbles_are_equivalent(df_complaints_by_city,
                              df_complaints_by_city_soln)

print("\n(Passed.)")
del df_complaints_by_city_soln

Let's use HoloViews+Bokeh to visualize the results as a stacked bar chart.

In [ ]:
# Let's consider only the top 25 complaints (by total)
top_complaints = df_complaint_freq[:25]
print("Top complaints:")
display(top_complaints)

In [ ]:
# Plot subset of data corresponding to the top complaints
df_plot = top_complaints.merge(df_complaints_by_city,
                               left_on=['type'],
                               right_on=['complaint_type'],
                               how='left')
df_plot.dropna(inplace=True)
print("Data to plot (first few rows):")
display(df_plot.head())
print("...")

Let's visualize this as a stacked bar chart. To do so, we'll need to reshape the data frame so that the values to be stacked appear as columns. It's the perfect application for cast!

In [ ]:
df_plot_stacked = cast(df_plot, key='city_name', value='complaint_count')
df_plot_stacked.fillna(0, inplace=True)
display(df_plot_stacked)

In [ ]:
# Some code to render a Bokeh stacked bar chart

kwargs_figure = {'title': "Distribution of the top 25 complaints among top 7 cities with the most complaints",
                 'width': 800,
                 'height': 400,
                 'tools': "hover,crosshair,pan,box_zoom,wheel_zoom,save,reset,help"}
p = make_stacked_bar(df_plot_stacked, 'complaint_type', TOP_CITIES, kwargs_figure=kwargs_figure)

p.xaxis.major_label_orientation = 0.66

from bokeh.models import HoverTool
hover_tool = p.select(dict(type=HoverTool))
hover_tool.tooltips = [("y", "$y{int}")]

show(p)

**Exercise 5** (2 points). The preceding code created a dataframe, `df_plot_stacked`, which was then used to create the stacked bar chart shown above.

Suppose we want to create a different stacked bar plot that shows, for each complaint type $t$ and city $c$, the fraction of all complaints of type $t$ that occurred in city $c$. Store your result in a dataframe named `df_plot_stacked_fraction`. It should have the same columns as `df_plot_stacked`.

> The test cell will create the chart in addition to checking your result. Note that the normalized bars will not necessarily add up to 1; why not?

> Note: The normalized bars will not necessarily add up to 1. Why not?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

df_plot_stacked_fraction.head()

In [ ]:
# Test cell: `norm_above_test`

if False:
    df_plot_stacked_fraction.to_csv('df_plot_stacked_fraction_soln.csv', index=False)

p = make_stacked_bar(df_plot_stacked_fraction, 'complaint_type', TOP_CITIES, kwargs_figure=kwargs_figure)

p.xaxis.major_label_orientation = 0.66

from bokeh.models import HoverTool
hover_tool = p.select(dict(type=HoverTool))
hover_tool.tooltips = [("y", "$y")]

show(p)

# Check numerical values
df_plot_stacked_fraction_soln = pd.read_csv('df_plot_stacked_fraction_soln.csv')
def merge_two_dicts(x, y):
    return {**x, **y}

def merge_many_dicts(dicts):
    x = {}
    for d in dicts:
        x = merge_two_dicts(x, d)
    return x

def tuple_sub(a, b):
    assert type(a) is tuple and type(b) is tuple and len(a) == len(b)
    return tuple(i - j for i, j in zip(a, b))

def all_tol(x, tol=1e-14):
    return all([abs(i) <= tol for i in x])

df_plot_stacked_fraction_soln = pd.read_csv('df_plot_stacked_fraction_soln.csv')
row_to_dict = lambda x: {x['complaint_type']: tuple(x[TOP_CITIES])}
your_soln = merge_many_dicts(df_plot_stacked_fraction.apply(row_to_dict, axis=1))
true_soln = merge_many_dicts(df_plot_stacked_fraction_soln.apply(row_to_dict, axis=1))
assert len(your_soln) == len(true_soln)
for key_true, value_true in true_soln.items():
    assert key_true in your_soln, "Your solution is missing the complaint type, '{}'".format(key_true)
    value_yours = your_soln[key_true]
    assert all_tol(tuple_sub(value_yours, value_true)), "Data for complaint '{}' of your solution differs from that of reference solution.".format(key_true)

print("\n(Passed!)")

## Dates and times in SQL

Recall that the input data had a column with timestamps corresponding to when someone submitted a complaint. Let's quickly summarize some of the features in SQL and Python for reasoning about these timestamps.

The `CreatedDate` column is actually a specially formatted date and time stamp, where you can query against by comparing to strings of the form, `YYYY-MM-DD hh:mm:ss`.

For example, let's look for all complaints on September 15, 2015.

In [ ]:
query = '''
  select ComplaintType, CreatedDate, City
    from data
    where CreatedDate >= "2015-09-15 00:00:00.0"
      and CreatedDate < "2015-09-16 00:00:00.0"
    order by CreatedDate
'''
df = pd.read_sql_query (query, disk_engine)
df

This next example shows how to extract just the hour from the time stamp, using SQL's `strftime()`.

In [ ]:
query = '''
  select CreatedDate, strftime ('%H', CreatedDate) as Hour, ComplaintType
    from data
    limit 5
'''
df = pd.read_sql_query (query, disk_engine)
df

**Exercise 6** (3 points). Construct a tibble called `df_complaints_by_hour`, which contains the total number of complaints during a given hour of the day. That is, the variables should be {`hour`, `count`} where each observation is the total number of complaints (`count`) that occurred during a given `hour`.

> Interpret `hour` as follows: when `hour` is `02`, that corresponds to the open time interval [`02:00:00`, `03:00:00.0`).

In [ ]:
# Your task: Construct `df_complaints_by_hour` as directed.
# YOUR CODE HERE
raise NotImplementedError()

# Displays your answer:
display(df_complaints_by_hour)

In [ ]:
# Test cell: `df_complaints_by_hour_test`
    
print ("Reading instructor's solution...")
if False:
    df_complaints_by_hour_soln.to_csv('df_complaints_by_hour_soln.csv', index=False)
df_complaints_by_hour_soln = pd.read_csv ('df_complaints_by_hour_soln.csv')
display (df_complaints_by_hour_soln)

df_complaints_by_hour_norm = df_complaints_by_hour.copy ()
df_complaints_by_hour_norm['hour'] = \
    df_complaints_by_hour_norm['hour'].apply (int)
assert tibbles_are_equivalent (df_complaints_by_hour_norm,
                               df_complaints_by_hour_soln)
print ("\n(Passed.)")

Let's take a quick look at the hour-by-hour breakdown above.

In [ ]:
%%opts Bars [width=640 height=320]
Bars(df_complaints_by_hour, kdims=['hour'], vdims=['count'])

An unusual aspect of these data are the excessively large number of reports associated with hour 0 (midnight up to but excluding 1 am), which would probably strike you as suspicious. Indeed, the reason is that there are some complaints that are dated but with no associated time, which was recorded in the data as exactly `00:00:00.000`.

In [ ]:
query = '''
  select count(*)
    from data
    where strftime ('%H:%M:%f', CreatedDate) = '00:00:00.000'
'''

pd.read_sql_query (query, disk_engine)

**Exercise 7** (2 points). What is the most common hour for noise complaints? Compute a tibble called `df_noisy_by_hour` whose variables are {`hour`, `count`} and whose observations are the number of noise complaints that occurred during a given `hour`. Consider a "noise complaint" to be any complaint string containing the word `noise`. Be sure to filter out any dates _without_ an associated time, i.e., a timestamp of `00:00:00.000`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

display(df_noisy_by_hour)

In [ ]:
# Test cell: `df_noisy_by_hour_test`

print ("Reading instructor's solution...")
if False:
    df_noisy_by_hour.to_csv('df_noisy_by_hour_soln.csv', index=False)
df_noisy_by_hour_soln = pd.read_csv ('df_noisy_by_hour_soln.csv')
display(df_noisy_by_hour_soln)

df_noisy_by_hour_norm = df_noisy_by_hour.copy()
df_noisy_by_hour_norm['hour'] = \
    df_noisy_by_hour_norm['hour'].apply(int)
assert tibbles_are_equivalent (df_noisy_by_hour_norm,
                               df_noisy_by_hour_soln)
print ("\n(Passed.)")

In [ ]:
%%opts Bars [width=640 height=320]
Bars(df_noisy_by_hour, kdims=['hour'], vdims=['count'])

**Exercise 8** (ungraded). Create a line chart to show the fraction of complaints (y-axis) associated with each hour of the day (x-axis), with each complaint type shown as a differently colored line. Show just the top 5 complaints (`top_complaints[:5]`). Remember to exclude complaints with a zero-timestamp (i.e., `00:00:00.000`).

> **Note.** This exercise is ungraded but we recommend spending some time giving it a try! Feel free to discuss your approaches to this problem on the discussion forums (but do try to do it yourself first).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Learn more

- Find more open data sets on [Data.gov](https://data.gov) and [NYC Open Data](https://nycopendata.socrata.com)
- Learn how to setup [MySql with Pandas and Plotly](http://moderndata.plot.ly/graph-data-from-mysql-database-in-python/)
- Big data workflows with [HDF5 and Pandas](http://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas)